In [1]:
# !/usr/bin/env python3
# Use other Python kernel other than the in ADS integrated one
# to provide more flexibility in the code
# and to avoid the limitation of the ADS integrated Python kernel

''' File: data_pre_processing.ipynb:
    This file is used to modify the generated data from data_gen.ipynb to finally match the format required by further DNN training by PyTorch.
'''

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter
import h5py
import os

In [ ]:
# Function Loading ...

def smooth_vec(y: np.ndarray) -> np.ndarray:
        '''use savgol_filter to smooth input vector'''
        n = y.size
        if n < 5:
            return y
        win = min(11, n if n % 2 == 1 else n - 1)
        win = max(win, 5)
        return savgol_filter(y, window_length=win, polyorder=2, mode='interp')


def error_gain_calc(meas_data_selected: np.ndarray, meas_data_smooth: np.ndarray):
    
    g1_matrix = np.matmul(meas_data_selected, meas_data_smooth.T)
    g2_matrix = np.matmul(meas_data_smooth, meas_data_smooth.T)
    g = np.sum(np.diag(g1_matrix)) / np.sum(np.diag(g2_matrix))
    error_gain = g - 1

    return error_gain, g


def alpha_beta_calc(Id_meas, Id_base, g,
                   n_bins=20, min_count=10,
                   log_bins=True, robust=False):

    Id_meas = np.asarray(Id_meas, float)
    Id_base = np.asarray(Id_base, float)
    if Id_meas.shape != Id_base.shape:
        raise ValueError("Id_meas 与 Id_base 形状必须一致")

    # 1) 残差与参考横轴
    ref = np.abs(g * Id_base).ravel()
    r   = (Id_meas - g * Id_base).ravel()

    good = np.isfinite(ref) & np.isfinite(r)
    ref = ref[good]; r = r[good]
    if ref.size < 2:
        raise ValueError("有效样本太少，无法拟合")

    # 2) 生成分箱（优先对数分箱；动态范围不足则退化为线性分箱）
    eps = 1e-15
    use_log = False
    if log_bins:
        pos = ref[ref > 0]
        if pos.size >= 10:
            lo, hi = np.percentile(pos, 5), np.percentile(pos, 95)
            if hi / max(lo, eps) >= 10:   # 至少一阶量级
                use_log = True
                lo = max(lo, eps)
                edges = np.logspace(np.log10(lo), np.log10(hi), n_bins + 1)
    if not use_log:
        lo, hi = np.percentile(ref, 5), np.percentile(ref, 95)
        edges = np.linspace(lo, hi, n_bins + 1)

    idx = np.digitize(ref, edges, right=False)  # 1..n_bins

    # 3) 每个箱的 σ（std 或 MAD）
    centers, sigmas, counts = [], [], []
    for b in range(1, len(edges)):
        m = (idx == b)
        c = int(np.count_nonzero(m))
        if c < min_count:
            continue
        center = np.median(ref[m])
        vals   = r[m]
        if robust:
            med = np.median(vals)
            sigma = 1.4826 * np.median(np.abs(vals - med))  # MAD→std
        else:
            sigma = vals.std(ddof=1)
        if np.isfinite(sigma) and sigma > 0:
            centers.append(center); sigmas.append(sigma); counts.append(c)

    centers = np.asarray(centers); sigmas = np.asarray(sigmas); counts = np.asarray(counts)
    if centers.size < 2:
        raise ValueError("有效分箱不足，调小 min_count 或增大 n_bins")

    # 4) 加权最小二乘拟合: sigma ≈ alpha * I + beta
    A = np.column_stack([centers, np.ones_like(centers)])
    w = np.sqrt(counts)                         # 权重≈置信度
    Aw = A * w[:, None]
    yw = sigmas * w
    (alpha, beta), *_ = np.linalg.lstsq(Aw, yw, rcond=None)

    # R^2 诊断
    yhat   = A @ np.array([alpha, beta])
    ss_res = float(np.sum((sigmas - yhat)**2))
    ss_tot = float(np.sum((sigmas - sigmas.mean())**2))
    r2     = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan

    diag = {
        "R2": r2,
        "centers": centers,
        "sigmas": sigmas,
        "counts": counts,
        "edges": edges,
        "used_log_bins": use_log,
    }
    return alpha, beta, diag




def noise_coefficient_estimation(meas_file: str, Vgs_value_num: int, Vds_value_num: int, Vds_range: tuple, Vds_step: float|int, channel_num: int, plot_flag = True, save_flag = True):
    '''
    Read meas data from .csv and return numpy, 
    also maps range into Vds in [-3.5, 8.5],
    with step size of 0.1,
    estimate 3 paramters in order to generate noise on simulated data
    '''

    # >> read data from measurement file
    df = pd.read_csv(meas_file)
    df.columns = ['Vgs', 'Vds', 'Ids'] # rename each column


    # >> validate dataset
    Vgs_unique_value = df['Vgs'].unique()
    Vgs_shape = set()
    for i in Vgs_unique_value:
        Vgs_shape.add(df[df['Vgs'] == i].shape)

    print(f'size of original measurement data is {len(Vgs_unique_value)}*{list(list(Vgs_shape)[0])[0]}')

    if (len(Vgs_shape) - 1) :
        print(f'【Error】sizes of different groups Vgs-bias are not matched with: {Vgs_shape}')
    else:
        print(f"sizes of different groups Vgs-bias are matched with: {Vgs_shape}")

    if (len(Vgs_unique_value)) != Vgs_value_num:
        raise ValueError(f'The number of different Vgs is given {Vgs_value_num} but is actually {len(Vgs_unique_value)} with Vgs = {Vgs_unique_value.tolist()}')


    # >> transform data to numpy and reshape + correct range
    # original range : Vds in [-5,20]
    #      new range : Vds in [-3.5,8.5]
    meas_data_selected = np.zeros((Vgs_value_num, Vds_value_num), dtype=np.float64)
    meas_data_smooth = meas_data_selected.copy()
    Vgs_unique_value.tolist().sort()
    for index, Vgs in enumerate(Vgs_unique_value):
        new_Vds_range = np.arange(Vds_range[0], Vds_range[1] + Vds_step, Vds_step)
        old_Vds_range = df[df['Vgs'] == Vgs]['Vds'].to_numpy()
        old_Ids_range = df[df['Vgs'] == Vgs]['Ids'].to_numpy()
        meas_data_selected[index,:] = np.interp(new_Vds_range, old_Vds_range, old_Ids_range)
        meas_data_smooth[index,:] = smooth_vec(meas_data_selected[index,:])


    # >> params regression - least squares scaling
    error_gain, g, alpha, beta = None, None, None, None

    # calculate error_gain
    error_gain, g = error_gain_calc(meas_data_selected, meas_data_smooth)

    # calculate alpha, beta
    meas_data_error = meas_data_selected - (g * meas_data_smooth)
    std_vector = np.sqrt(np.diag(np.matmul(meas_data_error.T, meas_data_error))/Vgs_value_num)

    alpha, beta = np.polyfit(new_Vds_range, std_vector, deg=1)

    alpha, beta, dialog = alpha_beta_calc( Id_meas=meas_data_selected,
                                        Id_base=meas_data_smooth,
                                        g = g ) # TODO: solve this part

    # optional, save measurement data with ruled format
    if save_flag:
        channel_iv = 0
        channel_gm = 1
        if channel_num == 2:
            meas_data_save = np.zeros((channel_num, Vgs_value_num - 1, Vds_value_num), dtype=np.float64)
            meas_data_save[channel_iv,:,:] = 0.5*(meas_data_selected[:-1,:] + meas_data_selected[1:,:])
            meas_data_save[channel_gm,:,:] = meas_data_selected[1:,:]- meas_data_selected[:-1,:]
        elif channel_num == 1:
            meas_data_save = np.zeros((Vgs_value_num, Vds_value_num), dtype=np.float64)
            meas_data_save[:] = meas_data_selected[:]
        else:
            raise ValueError(f'channel_num should be 1 or 2, but is {channel_num}')

        # check if dir exist
        split_index = meas_file[::-1].find('/')
        if channel_num == 2:
            path = meas_file[:(- split_index)] + 'meas_file_reshape.h5'
        elif channel_num == 1:
            path = meas_file[:(- split_index)] + 'meas_file_reshape_7row.h5'

        with h5py.File(path, 'w') as f:
            f.create_dataset('X', data=meas_data_save)
            print(f'sucessfully save measurement data under path {path}, with shape of {meas_data_selected.shape}')
        

    # optional, plot to check performance
    if plot_flag:

         # test plot 1 - original data vs imterploted data vs smoothed data
        _, ax = plt.subplots(2,2, figsize=(18, 13))
        for index, Vgs in enumerate(Vgs_unique_value):
            range1 = df['Vgs'] == Vgs
            range2 = df['Vds'] >= Vds_range[0]
            range3 = df['Vds'] <= Vds_range[1]
            range = range1 & range2 & range3
            # plot only within Vds range [-3.5,8.5]
            ax[0,0].plot(df[range]['Vds'], df[range]['Ids'], label = f"Vgs = {Vgs} V")
            ax[0,1].plot(new_Vds_range, meas_data_selected[index,:], label = f"Vgs = {Vgs} V")
            ax[1,0].plot(new_Vds_range, smooth_vec(meas_data_selected[index,:]), label = f"Vgs = {Vgs} V")
            ax[1,1].plot(new_Vds_range, meas_data_selected[index,:] - smooth_vec(meas_data_selected[index,:]), label = f"Vgs = {Vgs} V")

        ax[0,0].set_title('test plot of original meas data')
        ax[0,1].set_title('test plot of interplotted data')
        ax[1,0].set_title('test plot of smoothed data')
        ax[1,1].set_title('error between interplotted adn sommthed data')

        for i, j in np.ndindex((2,2)):
            ax[i,j].set_xlabel("VDS (V)")
            ax[i,j].set_ylabel("IDS (A)")
            ax[i,j].legend()
            ax[i,j].grid()
        plt.show()

        # test plot 2 - regression of alpha and beta
        _, ax = plt.subplots(figsize=(9, 6.5))
        ax.plot(new_Vds_range, std_vector, 'x', label = 'std of errors')
        ax.plot(new_Vds_range, new_Vds_range * alpha + beta,'--', label='regression curve')
        ax.set_title('test plot of original meas data')
        ax.set_xlabel("VDS (V)")
        ax.set_ylabel("Standard Deviation of IDS (A)")
        ax.legend()
        ax.grid()
         
    return error_gain, g, alpha, beta


# filter rule 1: max value of each sample shoudl be at least > 2, to remove meaningless noise
# filter rule 2: with risen Vgs, max Ids must also rise
def filter_mask(X: np.ndarray, mask_1_flag = False, mask_2_flag = False) -> np.ndarray:

    X = np.asarray(X)
    mask_2 = np.array([True]*X.shape[0]).reshape(-1,1)  # default all True
    mask_1 = mask_2.copy()
    # assert X.ndim == 3 and X.shape[1] == 7 and X.shape[2] == 121
    #  -> (N, 7)
    last_col = X[:, :, -1]

    if mask_1_flag:
        max_per_sample = np.max(X, axis=(1,2)).reshape(-1,1)
        mask_1 = (max_per_sample > 2)

    if mask_2_flag:
    # diff in direction of（7） -> (N, 6)，must be strictly increasing
        diffs = np.diff(last_col, axis=1)
        mask_2 = np.all(diffs > 0, axis=1).reshape(-1,1)

    mask = mask_1 & mask_2

    return mask

In [ ]:
'''STEP 1 : Variable INIT'''

# ---------------------------------PATH-------------------------

# path of saved .h5 data
# generated data path, the data adding noise will be saved in sub folder
data_save_path = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_gen/data/log_bigger_dataset"
noisy_data_path = data_save_path + '/noisy_data'

# path of measured data
meas_data_path = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_pre_processing/data"
meas_file_name = "meas_data.csv"
meas_file = meas_data_path + '/' + meas_file_name


# ----------------------------------VAR-------------------------

num_set = 100 # here only 10 process/dataset available
num_samples_single_set = 1000 # in each process/dataset 100 samples/simulations available
# in curret config, input dimension is 2*7*121
# channel 1: Ids-Vds curve
# channel 2: gm
# num_channel = 2  from now on, ignore the channel gm, use only channel Ids-Vds with row=7
num_channel = 1
channel_iv = 0
channel_gm = 1
row_X = 7
# col_X = 121
col_X = 186
row_Y = 11
col_Y = 1


Vgs_value_num = 7
# Vds_value_num = 121
Vds_value_num = 186
# Vds_range = (-3.5,8.5)
Vds_range = (-3.5,15)
Vds_step = 0.1


In [ ]:
'''STEP 2 : calculate Noise Coefficient'''

plot_validation_flag = False
validation_data_save_flag = True

error_gain, g, alpha, beta = noise_coefficient_estimation(meas_file, Vgs_value_num, Vds_value_num, Vds_range, Vds_step, num_channel, plot_validation_flag, validation_data_save_flag)


In [ ]:
'''STEP 3 : add noice in simulation data'''

# pre-define matrixs
X_iv = np.zeros((num_set*num_samples_single_set, row_X, col_X),dtype=np.float64)
X_iv_noisy = X_iv.copy()
X_noisy = np.zeros((num_set*num_samples_single_set, num_channel, row_X - 1, col_X),dtype=np.float64) # row minus 1 after size matching
Y = np.zeros((num_set*num_samples_single_set, row_Y, col_Y),dtype=np.float64)

# trans data from .h5 to numpy
for process_id in range(1, num_set + 1):
    with h5py.File(f"{data_save_path}\\dataset_process_{process_id}.h5", 'r') as f:
        X_iv[(process_id - 1)*num_samples_single_set : (process_id)*num_samples_single_set, :, :] = f['X_iv']
        Y[(process_id - 1)*num_samples_single_set : (process_id)*num_samples_single_set, :, :] = f['Y']


# add noise in simulation I-V data
sigma_matrix = np.abs(alpha * X_iv + beta)
noise = np.random.default_rng().normal(loc=0.0, scale=sigma_matrix)
X_iv_noisy = g * X_iv + noise
if num_channel == 2:
    X_noisy[:, channel_iv, :, :] = 0.5 * (X_iv_noisy[:, :-1,:] + X_iv_noisy[:, 1:,:]) # size match, use not original iv but its interplot
    X_noisy[:, channel_gm, :, :] = (X_iv_noisy[:, 1:,:] - X_iv_noisy[:, :-1,:])


# remove outliers
# filter rule 1: max value of each sample shoudl be at least > 2, to remove meaningless noise
# filter rule 2: with risen Vgs, max Ids must also rise
rule_1_flag = True
rule_2_flag = False
outliner_mask = filter_mask(X_iv_noisy, rule_1_flag, rule_2_flag)
X_iv_noisy_outlier_removal = X_iv_noisy[outliner_mask.reshape(-1),:,:]
Y_outlier_removal = Y[outliner_mask.reshape(-1),:,:]




In [ ]:
'''STEP x : Gen 2-stage dataset'''

dataset_1 = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_gen/data/bigger_dataset/noisy_data/train_bigger_dataset_filter_7row.h5"
dataset_2 = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_gen/data/normal_dataset/noisy_data/train_dataset_7row.h5"
dataset_3 = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_gen/data/log_normal_dataset/noisy_data/train_log_dataset_7row_2.h5"
dataset_4 = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_gen/data/log_normal_dataset/noisy_data/train_log_dataset_7row_1.h5"


f1 = h5py.File(dataset_1, 'r')
f2 = h5py.File(dataset_2, 'r')
f3 = h5py.File(dataset_3, 'r')
f4 = h5py.File(dataset_4, 'r')

print('f1-x shape: ',f1['X'].shape)
print('f2-x shape: ',f2['X'].shape)
print('f3-x shape: ',f3['X'].shape)
print('f4-x shape: ',f4['X'].shape)
print('f-x type: ',type(f3['X']))

f_X = np.vstack((f1['X'],f2['X'],f3['X']))
f_Y = np.vstack((f1['Y'],f2['Y'],f3['Y']))

meas_like_X = f4['X']
meas_like_Y = f4['Y']

print('f_X shape: ',f_X.shape)
print('f_Y shape: ',f_Y.shape)

print('meas_like_X shape: ',meas_like_X.shape)
print('meas_like_Y shape: ',meas_like_Y.shape)

In [ ]:
rule_1_flag = True
rule_2_flag = True
outliner_mask = filter_mask(f_X, rule_1_flag, rule_2_flag)
meas_like_outliner_mask = filter_mask(meas_like_X, rule_1_flag, rule_2_flag)

train_X = f_X[outliner_mask.reshape(-1),:,:]
train_Y = f_Y[outliner_mask.reshape(-1),:,:]

meas_like_X = meas_like_X[meas_like_outliner_mask.reshape(-1),:,:]
meas_like_Y = meas_like_Y[meas_like_outliner_mask.reshape(-1),:,:]

print('shape of full training set X: ',train_X.shape)
print('shape of full training set Y: ',train_Y.shape)

print('shape of meas like fine-turning set X: ',meas_like_X.shape)
print('shape of meas like fine-turning set Y: ',meas_like_Y.shape)


In [ ]:
full_data_save_path = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/dataset/training/stage_1_full_data.h5"
meas_like_data_save_path = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/dataset/training/stage_2_ml_data.h5"
full_plus_ml_data_save_path = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/dataset/training/stage_1_fpml_data.h5"

with h5py.File(full_data_save_path, 'w') as f:
    f.create_dataset('X', data=train_X)
    f.create_dataset('Y', data=train_Y)

with h5py.File(meas_like_data_save_path, 'w') as f:
    f.create_dataset('X', data=meas_like_X)
    f.create_dataset('Y', data=meas_like_Y)

with h5py.File(full_plus_ml_data_save_path, 'w') as f:
    f.create_dataset('X', data=np.vstack((train_X, meas_like_X)))
    f.create_dataset('Y', data=np.vstack((train_Y, meas_like_Y)))

f1.close()
f2.close()
f3.close()
f4.close()

In [ ]:
print(X_iv_noisy.shape)
print(Y.shape)


print(X_iv_noisy.shape, X_iv_noisy_outlier_removal.shape)
print(Y.shape, Y_outlier_removal.shape)
# print(np.where(outliner_mask==False)[0])
print(np.where(outliner_mask==False)[0].shape)


In [ ]:
'''STEP 4 : save noisy data as torch'''

# check if dir exist
if not os.path.exists(noisy_data_path):
    os.makedirs(noisy_data_path)
    print('dir not found, create new dir based on given path')
else:
    ('path exists')
print('saving data ...')

if num_channel == 2:
    file_name = 'train_dataset_6row.h5'
    path = noisy_data_path + '/' + file_name
    with h5py.File(path, 'w') as f:
        f.create_dataset('X', data=X_noisy)
        f.create_dataset('Y', data=Y)
elif num_channel == 1:
    file_name = 'train_bigger_dataset_7row.h5'
    path = noisy_data_path + '/' + file_name
    with h5py.File(path, 'w') as f:
        f.create_dataset('X', data=X_iv_noisy_outlier_removal)
        f.create_dataset('Y', data=Y_outlier_removal)
        # f.create_dataset('X', data=X_iv_noisy)
        # f.create_dataset('Y', data=Y)


print('data saved !')

In [ ]:
if num_channel == 1:
    file_name = 'train_bigger_dataset_7row.h5'
    path = noisy_data_path + '/' + file_name
    with h5py.File(path, 'r') as f:
        print(f['X'].shape)
    
    file_name = 'train_bigger_dataset_filter_7row.h5'
    path = noisy_data_path + '/' + file_name
    with h5py.File(path, 'r') as f:
        print(f['X'].shape)

In [ ]:
# test :: build a classifier to findout the data matching the distribution of meas data
# remove outliers
max_per_sample = np.max(X_iv_noisy_outlier_removal, axis=(1,2)).reshape(-1,1)
ratio_vector = np.zeros((max_per_sample.shape[0],1),dtype=np.float64)
for index in range(max_per_sample.shape[0]):
    ratio_vector[index] = np.mean(np.diff(X_iv_noisy_outlier_removal[index, :, -1],axis = 0)) / (max_per_sample[index, 0] / 6)

positive_ratio_index = ratio_vector > 0

print(f'ratio is {ratio_vector}')
print(f'positive ratio index is {np.where(positive_ratio_index==True)[0]}', f'with size of {np.where(positive_ratio_index==True)[0].shape}')
print(np.max(ratio_vector), np.where(ratio_vector==np.max(ratio_vector))[0])




In [ ]:
# test :: merge multiple different .h5 datasets

file_name_1 = 'E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_gen/data/log_bigger_dataset/noisy_data/train_bigger_dataset_7row_186col.h5'

file_name_2 = 'E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_gen/data/log_normal_dataset/noisy_data/train_log_dataset_7row_double.h5'

save_path = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/NN_training/dataset/training"

merge_file_name = 'train_log_full_bigger_dataset_7row.h5'

f1 = h5py.File(file_name_1, 'r')
f2 = h5py.File(file_name_2, 'r')

# match size
f_iterator = [f1,f2]
f_col = set(i['X'].shape[2] for i in f_iterator)
for index,f in enumerate(f_iterator):
    print(f"before correction: f{index+1}['X'].shape = {f['X'].shape}, f{index+1}['Y'].shape = {f['Y'].shape}")

# merge data
match_col = min(f_col)
if len(f_iterator) > 2:
    print(f'[Diag] col-num not matched, unified to {match_col}')
    data_x = np.vstack((f_iterator[0]['X'][:,:,:match_col], f_iterator[1]['X'][:,:,:match_col]))
    data_y = np.vstack((f_iterator[0]['Y'], f_iterator[1]['Y']))
    for f in f_iterator[2:]:
        data_x = np.vstack((data_x, f['X'][:,:,:match_col]))
        data_y = np.vstack((data_y, f['Y']))
else:
    data_x = np.vstack((f_iterator[0]['X'][:,:,:match_col], f_iterator[1]['X'][:,:,:match_col]))
    data_y = np.vstack((f_iterator[0]['Y'], f_iterator[1]['Y']))

print(f"after correction: data['X'].shape = {data_x.shape}, data['Y'].shape = {data_y.shape}")

# save data
path = save_path + '/' + merge_file_name
with h5py.File(path, 'w') as f:
    f.create_dataset('X', data=data_x)
    f.create_dataset('Y', data=data_y)

print('data saved !')
for f in f_iterator:
    f.close()

In [ ]:
meas_path = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_pre_processing/data/meas_file_reshape_7row.h5"
save_path = "E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data_pre_processing/data/temp.h5"

f = h5py.File(meas_path, 'r')
print(f.keys())
print(f['X'])

temp = np.array(f['X'][:]).reshape(1,7,121)
print(temp.shape)
f.close()

with h5py.File(save_path, 'w') as f:
    f.create_dataset('X', data=temp)
    print(f.keys())
    print(f['X'].shape)

In [ ]:
# test plot

index = 4000
index -= 1

legend_values = np.linspace(-3.5, 8.6, X_noisy.shape[3])
  
# create colormap
cmap = plt.cm.gist_ncar
norm = plt.Normalize(vmin=legend_values.min(), vmax=legend_values.max())

fig, ax = plt.subplots(1, 3, figsize=(20, 8)) 
for i in range(X_noisy.shape[2]):
    # plot 1
    color = cmap(norm(legend_values[i]))
    ax[0].plot(np.arange(-3.5, 8.6, 0.1), X_noisy[index, channel_iv, i, :])
    ax[0].grid(True)
    ax[0].set_title('noisy I-V')
    ax[0].set_xlabel("Vds (V)")
    ax[0].set_ylabel("Ids")

for i in range(X_noisy.shape[3]):
    # plot 2
    color = cmap(norm(legend_values[i]))
    ax[1].plot(np.arange(1.5, 7.5, 1), X_noisy[index, channel_iv, :, i], color=color)
    ax[1].grid(True)
    ax[1].set_title('noisy I-V')
    ax[1].set_xlabel("Vgs (V)")
    ax[1].set_ylabel("Ids")
    # plot 3
    color = cmap(norm(legend_values[i]))
    ax[2].plot(np.arange(1.5, 7.5, 1), X_noisy[index, channel_gm, :, i], color=color)
    ax[2].grid(True)
    ax[2].set_title('noisy gm')
    ax[2].set_xlabel("Vgs (V)")
    ax[2].set_ylabel("Ids")

# 加颜色条
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # 仅用于colorbar
cbar = fig.colorbar(sm, ax=ax)
cbar.set_label("Vds (V)")
    